In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_32.csv')
pred2 = dd.read_csv('test_pred_33.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 32 s, sys: 2.89 s, total: 34.9 s
Wall time: 7.61 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 212 ms, sys: 225 ms, total: 437 ms
Wall time: 435 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 281 µs, sys: 34 µs, total: 315 µs
Wall time: 320 µs


In [7]:
coef_lgb = 0.75
coef_mlp = 0.25

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 314 ms, sys: 538 ms, total: 852 ms
Wall time: 850 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000214,0.345268,0.000398,0.192891,0.141596,0.000456,0.094265,0.000255,0.001917,0.016579,0.002494,0.202730,0.000307,0.000629,0.245575
3492886,130787971,0.000223,0.011389,0.000242,0.050208,0.251989,0.000298,0.018038,0.291650,0.000399,0.022152,0.000946,0.348509,0.000283,0.003674,0.242776
3492887,130787974,0.000602,0.017150,0.000155,0.884417,0.021292,0.000361,0.060979,0.000065,0.000187,0.001149,0.000396,0.011493,0.000247,0.001506,0.084291
3492888,130788053,0.000086,0.723635,0.000138,0.128563,0.004101,0.000165,0.009331,0.096965,0.000482,0.000536,0.000301,0.034483,0.000143,0.001070,0.158408
3492889,130788054,0.000258,0.097851,0.000349,0.446554,0.114785,0.000785,0.051488,0.000556,0.001095,0.021297,0.002159,0.261155,0.000758,0.000910,0.225947


In [13]:
%%time
test_ensemble.to_csv('test_pred_35.csv',index=False)

CPU times: user 1min 30s, sys: 958 ms, total: 1min 31s
Wall time: 1min 31s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_35.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:08:47<00:00, 148kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification